In [36]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
folder = "D:\Research\FilaminC_rod\data\FLNCIg1to8(2269)\selected curves" # the folder where the data txt file saved
folder_name = Path(folder)
file = "20221213_2269_Spy_FLNCIg1to8_Avi_di100_nohandle_m270_PTB_37C_bead01_Cas200_01_1_10pns1_first7steps.txt" #the data txt file needed to choose

In [37]:
rawdata = np.loadtxt(folder_name/file, skiprows=2)
C = 200 #the C value of the tether

In [38]:
rawTime = rawdata[:,0]
rawMagPos = rawdata[:,1]
rawForce = C*(np.exp(-(13.5-rawMagPos)/0.36)+0.48*np.exp(-(13.5-rawMagPos)/1.12))
rawExt = rawdata[:,2]

MagPos_cutoff = 8.18 #input the starting magnet position for cutoff the cycle
#MagPos_cutoff = min(rawMagPos) #input the minimum magnet position and set as the cutoff of the cycle automatically
print(MagPos_cutoff)

8.18


In [39]:
cycleflag = 0
newTime = [[]]
newExt = [[]]
Force = [[]]
datalength = len(rawTime)
for i in range(datalength-1):
  if np.isnan(rawExt[i]):
    rawExt[i]=rawExt[i-1]
  if rawMagPos[i]>MagPos_cutoff:
    if all(rawMagPos[i+1:i+10]<=MagPos_cutoff):
      cycleflag = cycleflag + 1
      newTime.append([])
      newExt.append([])
      Force.append([])
    else:
        newTime[cycleflag].append(rawTime[i])
        newExt[cycleflag].append(rawExt[i])
        Force[cycleflag].append(rawForce[i])

cycle_data_increase = []
cycle_data_decrease = []
for j in range(len(Force)):
    max_index = np.argmax(Force[j])
    cycle_data_increase.append([newTime[j][:max_index], Force[j][:max_index], newExt[j][:max_index]]) 
    cycle_data_decrease.append([newTime[j][max_index:], Force[j][max_index:], newExt[j][max_index:]]) 

In [40]:
import zipfile
import os

zip_name = "test_zip" #change the zipfile name for saving
zip_file = folder+"\\"+zip_name


zip_file += ".zip"
with zipfile.ZipFile(zip_file, 'w') as zip_object:
    for i in range(len(cycle_data_increase)):
        txt_name = 'cycle_%.d_increase.txt'%(i+1)
        head_txt = 'Time\t' + 'Force\t' + 'Extension'
        np.savetxt(txt_name, np.transpose(cycle_data_increase[i]), header=head_txt, comments='')
        zip_object.write(txt_name)
        os.remove(txt_name)
    
    for j in range(len(cycle_data_decrease)):
        txt_name = 'cycle_%.d_decrease.txt'%(j+1)
        head_txt = 'Time\t' + 'Force\t' + 'Extension'
        np.savetxt(txt_name, np.transpose(cycle_data_decrease[j]), header=head_txt, comments='')
        zip_object.write(txt_name)
        os.remove(txt_name)